In [3]:
"sdf~".encode("ascii", "ignore").decode("ascii")

'sdf~'

In [4]:
# check if a string is ascii
def is_ascii(s):
    return all(ord(c) < 128 for c in s)


is_ascii("~")

True

In [10]:
import requests
from bs4 import BeautifulSoup


# function to get the AV star's name based on the video id
def get_name(video_id):
    # construct the url
    url = f"https://www.javlibrary.com/cn/vl_searchbyid.php?keyword={video_id}"
    print(url)
    # send a GET request to the url
    response = requests.get(url)
    # print(response)
    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    # print(soup)
    # find the div element that contains the AV star's name
    av_star_div = soup.find("span", {"class": "star"})
    print(av_star_div)
    # extract the AV star's name from the div element
    av_star_name = av_star_div.text.strip()
    return av_star_name


# get_name('ssis 640')

for n in ["ssis 640", "ssis 641", "rbd356", "huntb102"]:
    print(get_name(n))

https://www.javlibrary.com/cn/vl_searchbyid.php?keyword=ssis 640
<span class="star"><a href="vl_star.php?s=aehvu" rel="tag">七ツ森りり</a></span>
七ツ森りり
https://www.javlibrary.com/cn/vl_searchbyid.php?keyword=ssis 641
<span class="star"><a href="vl_star.php?s=aexvs" rel="tag">有栖花あか</a></span>
有栖花あか
https://www.javlibrary.com/cn/vl_searchbyid.php?keyword=rbd356
<span class="star"><a href="vl_star.php?s=kqia" rel="tag">愛花沙也</a></span>
愛花沙也
https://www.javlibrary.com/cn/vl_searchbyid.php?keyword=huntb102
<span class="star"><a href="vl_star.php?s=azmbm" rel="tag">あおいれな</a></span>
あおいれな


In [10]:
import os
import re
import datetime

# define the directory path
directory = "U:"
directories =["t:","x:","m:"]
# directories =["m:","k:","l:","r:","s:","t:","n:","o:","p:","q:","u:","v:","w:"]
# regex that matches the string like 'ssis-345.mp4','ssq-234.mp4','hunbg 234.mp4','asd 234c.mp4'
regex = r"^[a-zA-Z]{2,5}[- ]?\d{2,3}(\D+\d*\D*){0,1}\.mp4$"

def remove_file_name_suffix(file, root):
    if file.endswith(".mp4"):
        file_path = os.path.join(root, file)
        new_file_name = re.sub(r"(\d+).*", r"\1", file)
        return new_file_name


def set_file_last_modified(file_path, dt):
    dt_epoch = dt.timestamp()
    os.utime(file_path, (dt_epoch, dt_epoch))


import requests
from bs4 import BeautifulSoup


# function to get the AV star's name based on the video id
def get_name(video_id):
    # construct the url
    url = f"https://www.javlibrary.com/cn/vl_searchbyid.php?keyword={video_id}"
    # send a GET request to the url
    response = requests.get(url)
    # print(response)
    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    # print(soup)
    av_star_div = soup.find("span", {"class": "star"})
    if av_star_div:
        # extract the AV star's name from the div element
        av_star_name = av_star_div.text.strip()
        print(av_star_name)
    else:
        av_star_name = None
        divTag = soup.find_all("div", {"class": "video"})
        if len(divTag) > 0:
            for v in divTag:
                # print(v)
                # print(v.find("div", {"class": "id"}).text)
                # print(v.find("a").get("href"))
                if (
                    v.find("div", {"class": "id"}).text.replace("-", "").lower()
                    == video_id.replace("-", "").lower()
                ):
                    vurl = f"https://www.javlibrary.com/cn/{v.find('a').get('href').replace(f'./','')}"
                    print(vurl)
                    res = requests.get(vurl)
                    so = BeautifulSoup(res.content, "html.parser")
                    av_star_div = so.find("span", {"class": "star"})
                    print(av_star_div)
                    if av_star_div:
                        # extract the AV star's name from the div element
                        av_star_name = av_star_div.text.strip()
                        print(av_star_name)
                        if av_star_name:
                            break
    return av_star_name


def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def has_name(my_array, my_string):
    for element in my_array:
        if element in my_string:
            return True

    return False
# loop through all files and directories in the directory recursively
n = 0
def modify_new_video(d, el):
    for root, dirs, files in os.walk(d):
        for file in files:
            #print(file)
            # check if the file name starts with "test"
            if is_ascii(file) and re.match(regex, file) and "无名女优" not in file and not has_name(el, file):
                print(file)
                # create the new file name by replacing "test" with "haola"
                pure_name = remove_file_name_suffix(file, root)
                star_name = get_name(pure_name)
                
                file_path = os.path.join(root, file)
                if star_name is None:
                    star_name = "无名女优"
                if star_name in file:
                    el.append(star_name)

                    continue
                new_file_path = os.path.join(root, file.replace(".mp4", star_name + ".mp4"))
                os.rename(file_path, new_file_path)
                now = datetime.datetime.now()
                #set_file_last_modified(new_file_path, now)
                print(file_path + " ----------> " + new_file_path)
            #     break
def modify_nofound_video(d):
    
    for root, dirs, files in os.walk(d):
        for file in files:
            # check if the file name starts with "test"
            if re.match(regex, file) and "无名女优" in file:
                # create the new file name by replacing "test" with "haola"
                pure_name = remove_file_name_suffix(file, root)
                star_name = get_name(pure_name)
                print(star_name)
                file_path = os.path.join(root, file)
                if star_name is None:
                    star_name = "无名女优"
                if star_name in file:
                    print(el)
                    continue
                new_file_path = os.path.join(root, file.replace("无名女优.mp4", star_name + ".mp4"))
                os.rename(file_path, new_file_path)
                now = datetime.datetime.now()
                set_file_last_modified(new_file_path, now)
                print(file_path + " -> " + new_file_path)
def rename_video(d, pres):
    for root, dirs, files in os.walk(d):
        for file in files:
            # check if the file name starts with "test"
            for pre in pres:
                if pre  in file:

             
                    # create the new file name by replacing "test" with "haola"
                    new_file_path = os.path.join(root, file.replace(pre, ""))
                    file_path = os.path.join(root, file)
                    os.rename(file_path, new_file_path)
                    # now = datetime.datetime.now()
                    # set_file_last_modified(new_file_path, now)
                    print(file_path + " -> " + new_file_path)                
def all_directories(el):
    for d in directories:
        pres=['18bt.net_','@江南@jnty60.app-']
        rename_video(d, pres)
        modify_new_video(d, el)
el = ['Rio', 'RYU', 'AIKA', 'RYU', 'SARA', 'Rio', 'YURI', 'JULIA', 'JULIA', 'JULIA', 'JULIA', 'NAOMI', 'ANRI', 'NAOMI', 'Maika', 'Nina', 'Marin.', 'RYU', 'KAORI', 'Lucy heart', 'JULIA', 'JULIA', 'KAORI', 'CHIYU', 'miru', 'MINAMO', 'MINAMO', 'MINAMO']        

try:
    all_directories(el)
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
finally:
    print(el)



FOCS-147C.mp4
東山みえ
t:xunlei\FOCS-147C.mp4 ----------> t:xunlei\FOCS-147C東山みえ.mp4
FSDSS-634C.mp4
吉高寧々
t:xunlei\FSDSS-634C.mp4 ----------> t:xunlei\FSDSS-634C吉高寧々.mp4
FSDSS-636C.mp4
小野夕子
t:xunlei\FSDSS-636C.mp4 ----------> t:xunlei\FSDSS-636C小野夕子.mp4
FSDSS-638C.mp4
楓カレン
t:xunlei\FSDSS-638C.mp4 ----------> t:xunlei\FSDSS-638C楓カレン.mp4
HMN-450C.mp4
東條なつ
t:xunlei\HMN-450C.mp4 ----------> t:xunlei\HMN-450C東條なつ.mp4
HMN-455C.mp4
響蓮
t:xunlei\HMN-455C.mp4 ----------> t:xunlei\HMN-455C響蓮.mp4
JUFE-495C.mp4
五日市芽依
t:xunlei\JUFE-495C.mp4 ----------> t:xunlei\JUFE-495C五日市芽依.mp4
JUQ-351C.mp4
愛弓りょう
t:xunlei\JUQ-351C.mp4 ----------> t:xunlei\JUQ-351C愛弓りょう.mp4
JUQ-352C.mp4
栗山莉緒
t:xunlei\JUQ-352C.mp4 ----------> t:xunlei\JUQ-352C栗山莉緒.mp4
JUQ-360C.mp4
並木あいな
t:xunlei\JUQ-360C.mp4 ----------> t:xunlei\JUQ-360C並木あいな.mp4
JUQ-392C.mp4
水戸かな
t:xunlei\JUQ-392C.mp4 ----------> t:xunlei\JUQ-392C水戸かな.mp4
MEYD-835C.mp4
天海つばさ
t:xunlei\MEYD-835C.mp4 ----------> t:xunlei\MEYD-835C天海つばさ.mp4
MEYD-841C.mp4
饭冈加奈子
t:xunlei\MEYD

In [49]:
# Assuming the string is stored in the variable 'my_string' and the array is stored in the variable 'my_array'
# We can use a loop to iterate over each element in the array and check if it is present in the string
# If any element is present in the string, we c[0,1]an return True, else we can return False
regex = r"^[a-zA-Z]{2,5}[- ]?\d{2,3}(\D+\d*\D*){0,1}\.mp4$"
print(re.match(regex, "NSFS-205_fasdf.mp4") )


<re.Match object; span=(0, 18), match='NSFS-205_fasdf.mp4'>


In [41]:
import requests
from bs4 import BeautifulSoup
import json


# function to get the AV star's name based on the video id
def get_name(video_id):
    # construct the url
    url = f"https://www.javlibrary.com/cn/vl_searchbyid.php?keyword={video_id}"
    # send a GET request to the url
    response = requests.get(url)
    # print(response)
    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    # print(soup)
    av_star_div = soup.find("span", {"class": "star"})
    if av_star_div:
        # extract the AV star's name from the div element
        av_star_name = av_star_div.text.strip()
        print(av_star_name)
    else:
        av_star_name = None
        divTag = soup.find_all("div", {"class": "video"})
        if len(divTag) > 0:
            for v in divTag:
                # print(v)
                # print(v.find("div", {"class": "id"}).text)
                # print(v.find("a").get("href"))
                if (
                    v.find("div", {"class": "id"}).text.replace("-", "").lower()
                    == video_id.replace("-", "").lower()
                ):
                    vurl = f"https://www.javlibrary.com/cn/{v.find('a').get('href').replace(f'./','')}"
                    print(vurl)
                    res = requests.get(vurl)
                    so = BeautifulSoup(res.content, "html.parser")
                    av_star_div = so.find("span", {"class": "star"})
                    print(av_star_div)
                    if av_star_div:
                        # extract the AV star's name from the div element
                        av_star_name = av_star_div.text.strip()
                        print(av_star_name)
                        if av_star_name:
                            break
    return av_star_name
    # for tag in divTag:
    #     tdTags = tag.find_all("td", {"class": "align-right"})
    #     for tag in tdTags:
    #         print tag.text
    # print(divTag[0].get('id'))
    # print(divTag[0].get('title'))


get_name()

<div class="video" id="vid_javli6j6qa"><a href="./?v=javli6j6qa" title="IPX-402 絶頂117回！大痙攣141回！潮吹き8100cc！エロス極限突破トランス絶頂FUCK 相沢みなみ （ブルーレイディスク）"><div class="id">IPX-402</div><img border="0" height="200" onerror="ThumbError(this, 'https://t36.pixhost.to/thumbs/106/126047963_t414660.jpg');" src="https://pics.dmm.co.jp/mono/movie/adult/9ipx402/9ipx402ps.jpg" width="147"><div class="title">絶頂117回！大痙攣141回！潮吹き8100cc！エロス極限突破トランス絶頂FUCK 相沢みなみ （ブルーレイディスク）</div></img></a><div class="toolbar"><a class="icn_want" id="javli6j6qa" title="我想要"></a><a class="icn_seen" id="javli6j6qa" title="看过了"></a><a class="icn_have" id="javli6j6qa" title="已拥有"></a></div></div>
IPX-402
./?v=javli6j6qa
https://www.javlibrary.com/cn/?v=javli6j6qa
<span class="star"><a href="vl_star.php?s=afgda" rel="tag">相沢みなみ</a></span>
相沢みなみ


'相沢みなみ'

In [60]:
def is_japanese(string):
    return bool(re.search(r"[\u3040-\u30ff\u4e00-\u9fff\uf900-\ufaff]", string))


print(is_japanese("asdfa"))
print(is_japanese("の——うた"))
print(is_japanese("好"))

False
True
True


In [46]:
import re

# define the directory path


# regex that matches the string like 'ssis-345.mp4','ssq-234.mp4','hunbg 234.mp4','asd 234c.mp4'
def remove_file_name_suffix(file):
    if file.endswith(".mp4"):
        new_file_name = re.sub(r"(\d+).*", r"\1", file)
        print(new_file_name)


remove_file_name_suffix("afd444武名.mp4")

afd444
